In [1]:
import pandas as pd
import os
import re
import sys
import numpy as np
from merf import MERF
import matplotlib.pyplot as plt
import seaborn as sns
import itertools 
sns.set_context("poster")
from sklearn.ensemble import RandomForestRegressor
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (11,8)
from merf.merf import MERF
from sklearn.model_selection import train_test_split, KFold
from merf.viz import plot_merf_training_stats

current_dir = os.getcwd() # Get the current working directory
parent_dir = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(parent_dir)
from em_utils import *

# Create output directory if it doesn't exist
output_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_plots/5.combined"
df_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/play_scripts/2.models/merf_python/merf_dfs/5.combined"
os.makedirs(output_dir, exist_ok=True)

# Read in prior DFs

In [2]:
# Meta / clinical
clin_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/clinical/transformed/aim2/merf_ready/"
test_long = pd.read_csv(os.path.join(clin_dir, 'test_clinical_no_na.csv'))  
train_long = pd.read_csv(os.path.join(clin_dir, 'train_clinical_no_na.csv'))  
full_long = pd.read_csv(os.path.join(clin_dir, 'full_train_clinical.csv'))  
raw_train = pd.read_csv(os.path.join(clin_dir, 'raw__train_clinical.csv'))  
raw_test = pd.read_csv(os.path.join(clin_dir, 'raw_test_clinical.csv'))

print("---------- Read metadata ----------")
m1_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/clinical/transformed/aim2"
#met_test = read_data(m1_dir, "long_a2_test_samples_standard_clinical_feb20.csv")
#met_train = read_data(m1_dir, "long_a2_train_samples_standard_clinical_feb20.csv")
met_test = read_data(m1_dir, "long_a2_test_samples_extra_clinical_feb20.csv")
met_train = read_data(m1_dir, "long_a2_train_samples_extra_clinical_feb20.csv")
#met_full = read_data(m1_dir, "a2_meta_Transformed_standard_clinical.csv")
#met_full_raw = read_data(m1_dir, "a2_meta_not_Transformed_standard_clinical.csv")

# Genetic 
gen_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/genetic/"
gen_test = pd.read_csv(os.path.join(gen_dir, 'genetic_risk_testing.csv'))  
gen_train = pd.read_csv(os.path.join(gen_dir, 'genetic_risk_training.csv')) 

# Taxa
tax_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/taxa/aim2_transformed/genus/"
test_tax_no_na = pd.read_csv(os.path.join(tax_dir, 'aim2_clr_testing_feb20.csv'))
train_tax_no_na = pd.read_csv(os.path.join(tax_dir, 'aim2_clr_training_feb20.csv'))
#full_train_tax = pd.read_csv(os.path.join(tax_dir, 'full_train_tax.csv'))
#full_test_tax = pd.read_csv(os.path.join(tax_dir, 'full_test_tax.csv'))

# Functional
func_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/functional/aim2/"
test_func_no_na = pd.read_csv(os.path.join(func_dir, 'all_clr_testing_feb20.csv'))  # Read test_tax_no_na
train_func_no_na = pd.read_csv(os.path.join(func_dir, 'all_clr_training_feb20.csv'))  # Read train_tax_no_na
#full_train_func = pd.read_csv(os.path.join(func_dir, 'full_train_functional.csv'))  # Read full_train_tax
#full_test_func = pd.read_csv(os.path.join(func_dir, 'full_test_functional.csv'))

# Micom fluxes
micom_dir = "/Users/emily/projects/research/Stanislawski/comps/mutli-omic-predictions/data/micom/aim2/"
test_micom_no_na = pd.read_csv(os.path.join(micom_dir, 'flux_all_testing_feb20.csv'))  # Read in test_micom_no_na
train_micom_no_na = pd.read_csv(os.path.join(micom_dir, 'flux_all_training_feb20.csv'))  # Read in train_micom_no_na
#full_train_micom = pd.read_csv(os.path.join(micom_dir, 'full_train_micom.csv'))  # Read in full_train_micom
#full_test_micom = pd.read_csv(os.path.join(micom_dir, 'full_test_micom.csv')) 

---------- Read metadata ----------


In [3]:
print("meta test = ", met_train.shape)
print("Meta tst columns", met_train.columns.to_list())

print("meta train shape = ", met_test.shape)
print("Meta train df columns", met_test.columns.to_list())

print("genetic df shape = ", gen_train.shape)
print("Genetic df columns", gen_train.columns.to_list())
print("Taxa df shape = ", train_tax_no_na.shape)
print("Taxa df columns", train_tax_no_na.columns.to_list())
print("Functional df shape = ", train_func_no_na.shape)
print("Functional df columns", train_func_no_na.columns.to_list())
print("Micom df shape = ", train_micom_no_na.shape)
print("Micom df columns", train_micom_no_na.columns.to_list())

meta test =  (291, 20)
Meta tst columns ['Unnamed: 0', 'record_id', 'subject_id', 'randomized_group', 'cohort_number', 'sex', 'race', 'age', 'outcome_BMI_fnl', 'Glucose', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid', 'Peptide_YY', 'Ghrelin', 'Leptin', 'Hemoglobin_A1C', 'time']
meta train shape =  (72, 20)
Meta train df columns ['Unnamed: 0', 'record_id', 'subject_id', 'randomized_group', 'cohort_number', 'sex', 'race', 'age', 'outcome_BMI_fnl', 'Glucose', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid', 'Peptide_YY', 'Ghrelin', 'Leptin', 'Hemoglobin_A1C', 'time']
genetic df shape =  (120, 10)
Genetic df columns ['Unnamed: 0', 'subject_id', 'bmi_prs', 'old_or_new', 'predicted_BL_BMI', 'differences_BL_BMI', 'diff_BMI_quartile', 'diff_BMI_std', 'methyl_bmi_rs', 'methyl_bmi_rs_standardized']
Taxa df shape =  (461, 183)
Taxa df columns ['Unnamed: 0', 'g__Parabacteroides_B_862066', 'g__Coprenecus', 'g_

In [4]:
print("meta subject_id values:", met_train['subject_id'])
print("meta time values:", met_train['time'])

meta subject_id values: 0      LFI-003
1      ROL-006
2      AKE-009
3      HGI-010
4      AKI-011
        ...   
286    SKA-195
287    KHU-196
288    LPF-198
289    KEL-199
290    BIN-201
Name: subject_id, Length: 291, dtype: object
meta time values: 0       0
1       0
2       0
3       0
4       0
       ..
286    12
287    12
288    12
289    12
290    12
Name: time, Length: 291, dtype: int64


In [5]:
print("genetic subject_id values:", gen_train['subject_id'])
print("unnamed column in the gennetic data: ", gen_train['Unnamed: 0'])

genetic subject_id values: 0      ABR-079
1      ACO-053
2      ADA-105
3      AGA-071
4      AHE-055
        ...   
115    TSH-146
116    TSL-056
117    VCA-041
118    YOR-103
119    YSU-097
Name: subject_id, Length: 120, dtype: object
unnamed column in the gennetic data:  0        1
1        2
2        3
3        4
4        5
      ... 
115    116
116    117
117    118
118    119
119    120
Name: Unnamed: 0, Length: 120, dtype: int64


In [6]:
print("taxa all_samples values:", train_tax_no_na['all_samples'])

taxa all_samples values: 0       79
1       79
2       53
3       53
4       53
      ... 
456    103
457     97
458     97
459     97
460     97
Name: all_samples, Length: 461, dtype: int64


In [7]:
print("functional all_samples values:", train_func_no_na['all_samples'])
print("functional time values:", train_func_no_na['time'])
print("functional SampleID values:", train_func_no_na['SampleID'])

functional all_samples values: 0      105
1       71
2       71
3        9
4        9
      ... 
346     56
347     41
348     97
349     72
350     38
Name: all_samples, Length: 351, dtype: int64
functional time values: 0       0.0
1      12.0
2       6.0
3      12.0
4       6.0
       ... 
346     6.0
347     NaN
348     0.0
349     0.0
350     0.0
Name: time, Length: 351, dtype: float64
functional SampleID values: 0       ADA-105.BL
1      AGA-071.12m
2       AGA-071.6m
3      AKE-009.12m
4       AKE-009.6m
          ...     
346     TSL-056.6m
347    VCA-041.18m
348     YSU-097.BL
349     KRI-072.BL
350     KVA-038.BL
Name: SampleID, Length: 351, dtype: object


In [8]:
print("Micom all_samples values:", train_micom_no_na['all_samples'])
print("Micom time values:", train_micom_no_na['time'])
print("Micom sample_id values:", train_micom_no_na['sample_id'])

Micom all_samples values: 0      130
1      174
2       86
3       17
4       41
      ... 
282    166
283     67
284    113
285     43
286    149
Name: all_samples, Length: 287, dtype: int64
Micom time values: 0       0
1       0
2       0
3       0
4       0
       ..
282    12
283    12
284    12
285    12
286    12
Name: time, Length: 287, dtype: int64
Micom sample_id values: 0       EBE-130.BL
1       BSA-174.BL
2       TDU-086.BL
3       LVA-017.BL
4       VCA-041.BL
          ...     
282    QNG-166.12m
283    NDI-067.12m
284    TRO-113.12m
285    SCA-043.12m
286    NPO-149.12m
Name: sample_id, Length: 287, dtype: object


### Merge Micom and Functional data

Micom all_samples column matches Functional Sample.ID
##### For training

In [9]:
### Convert data types for merging
train_micom_no_na['sample_id'] = train_micom_no_na['sample_id'].astype(str)  # Convert to string
train_func_no_na['SampleID'] = train_func_no_na['SampleID'].astype(str)  # Ensure SampleID is string

print("Micom sample_id values:", train_micom_no_na['sample_id'])
print("Functional SampleID values:", train_func_no_na['SampleID']) 

# Count the number of matching rows between 'sample_id' and 'SampleID'
matching_rows_count = train_micom_no_na['sample_id'].isin(train_func_no_na['SampleID']).sum()
print("Number of matching rows between Micom sample_id and Functional SampleID:", matching_rows_count)

Micom sample_id values: 0       EBE-130.BL
1       BSA-174.BL
2       TDU-086.BL
3       LVA-017.BL
4       VCA-041.BL
          ...     
282    QNG-166.12m
283    NDI-067.12m
284    TRO-113.12m
285    SCA-043.12m
286    NPO-149.12m
Name: sample_id, Length: 287, dtype: object
Functional SampleID values: 0       ADA-105.BL
1      AGA-071.12m
2       AGA-071.6m
3      AKE-009.12m
4       AKE-009.6m
          ...     
346     TSL-056.6m
347    VCA-041.18m
348     YSU-097.BL
349     KRI-072.BL
350     KVA-038.BL
Name: SampleID, Length: 351, dtype: object
Number of matching rows between Micom sample_id and Functional SampleID: 287


64 samples are in the pathway dataset that are not in the micom dataset 

In [10]:
### Merge Micom and Functional data
merged_micom_func = pd.merge(train_micom_no_na, train_func_no_na, 
                     left_on='sample_id', right_on='SampleID', 
                     how='inner')
print("Merged micom_func shape: ", merged_micom_func.shape)
print("Merged micom_func columns: ", merged_micom_func.columns.to_list())
print("Merged micom_func id: ", merged_micom_func['sample_id'])
print("Merged micom_func all_samples: ", merged_micom_func['all_samples_y'])

Merged micom_func shape:  (287, 365)
Merged micom_func columns:  ['Unnamed: 0_x', 'sample_id', 'proton', 'Diacetyl', 'Butyrate', 'Formate', 'acetone', 'X2.methylbutyrate', 'Valeric.Acid', 'Methane', 'X1.4.Dihydroxy.2.naphthoate', 'X2.Oxoglutarate', 'Pyruvate', 'D.lactate', 'L.lactate', 'laurate', 'Phenol', 'L.Carnitine', 'Isopropanol', 'p.Cresol', 'O2', 'X1.Propanol', 'butanol', 'Propanal', 'X4.hydroxyphenylacetate', 'X.S..propane.1.2.diol', 'Propane.1.3.diol', 'R_R__2_3_Butanediol', 'Oxaloacetate', 'sulfate', 'octadecenoate..n.C18.1.', 'o.Succinylbenzoate', 'Thymidine', 'Reduced.riboflavin', 'Riboflavin', 'X2.Hydroxyphenylacetate', 'Sulfite', 'thiosulfate.2..', 'X.3.4.dihydroxyphenyl.acetate', 'Diphosphate', 'trimethylamine', 'glycine.betaine', 'Choline', 'Trimethylamine.N.oxide', 'Isochorismate', 'Inosine', 'xanthosine', 'Hypoxanthine', 'Xanthine', 'uracil', 'Uridine', 'Methanethiol', 'keto.phenylpyruvate', 'D.Arabinose', 'D.Galactose', 'Nicotinate', 'chorismate', 'tetradecanoate..n.

##### For testing

In [11]:
### Convert data types for merging
test_micom_no_na['sample_id'] = test_micom_no_na['sample_id'].astype(str)  # Convert to string
test_func_no_na['SampleID'] = test_func_no_na['SampleID'].astype(str)  # Ensure SampleID is string

print("Micom sample_id values:", test_micom_no_na['sample_id'])
print("Functional SampleID values:", test_func_no_na['SampleID']) 

# Count the number of matching rows between 'sample_id' and 'SampleID'
matching_rows_count = test_micom_no_na['sample_id'].isin(test_func_no_na['SampleID']).sum()
print("Number of matching rows between Micom sample_id and Functional SampleID:", matching_rows_count)

### Merge Micom and Functional data for testing
test_merged_micom_func = pd.merge(test_micom_no_na, test_func_no_na, 
                     left_on='sample_id', right_on='SampleID', 
                     how='inner')
print("Merged micom_func shape: ", test_merged_micom_func.shape)

Micom sample_id values: 0      JPO-022.BL
1      BHO-014.BL
2      JJO-093.BL
3      COW-066.BL
4      NMO-151.BL
         ...     
65    WPE-005.12m
66    CWA-161.12m
67    BHO-014.12m
68    FWI-098.12m
69    AAL-144.12m
Name: sample_id, Length: 70, dtype: object
Functional SampleID values: 0     AAL-144.12m
1      AAL-144.6m
2      AME-157.6m
3     BHO-014.12m
4      BHO-014.6m
         ...     
78     NKA-090.BL
79     SKA-087.BL
80    WPE-005.18m
81     ZVU-096.BL
82     JDI-140.BL
Name: SampleID, Length: 83, dtype: object
Number of matching rows between Micom sample_id and Functional SampleID: 70
Merged micom_func shape:  (70, 365)


13 samples in functional id test set is not in micom test set 

### Merge Micom and Functional data with Taxa data 

Using all_samples and time somehow

For training: 

In [12]:
### Merge merged_micom_func with taxa_train

micom_func_tax = pd.merge(merged_micom_func, train_tax_no_na, 
                         left_on=['all_samples_y', 'time_y'], 
                         right_on=['all_samples', 'time'], 
                         how='inner')

print("Final merged shape: ", micom_func_tax.shape)
print("Micom, taxa and functional df sample_id: ", micom_func_tax['sample_id'])
print("Micom, taxa and functional df all_sampls: ", micom_func_tax['all_samples'])

Final merged shape:  (299, 548)
Micom, taxa and functional df sample_id:  0       EBE-130.BL
1       BSA-174.BL
2       TDU-086.BL
3       LVA-017.BL
4       VCA-041.BL
          ...     
294    QNG-166.12m
295    NDI-067.12m
296    TRO-113.12m
297    SCA-043.12m
298    NPO-149.12m
Name: sample_id, Length: 299, dtype: object
Micom, taxa and functional df all_sampls:  0      130
1      174
2       86
3       17
4       41
      ... 
294    166
295     67
296    113
297     43
298    149
Name: all_samples, Length: 299, dtype: int64


299 all sample variables (duplicates included)

For testig: 

In [13]:
test_micom_func_tax = pd.merge(test_merged_micom_func, test_tax_no_na, 
                         left_on=['all_samples_y', 'time_y'], 
                         right_on=['all_samples', 'time'], 
                         how='inner')

In [14]:
print("Final merged columns: ", test_micom_func_tax.columns.to_list())

Final merged columns:  ['Unnamed: 0_x', 'sample_id', 'proton', 'Diacetyl', 'Butyrate', 'Formate', 'acetone', 'X2.methylbutyrate', 'Valeric.Acid', 'Methane', 'X1.4.Dihydroxy.2.naphthoate', 'X2.Oxoglutarate', 'Pyruvate', 'D.lactate', 'L.lactate', 'laurate', 'Phenol', 'L.Carnitine', 'Isopropanol', 'p.Cresol', 'O2', 'X1.Propanol', 'butanol', 'Propanal', 'X4.hydroxyphenylacetate', 'X.S..propane.1.2.diol', 'Propane.1.3.diol', 'R_R__2_3_Butanediol', 'Oxaloacetate', 'sulfate', 'octadecenoate..n.C18.1.', 'o.Succinylbenzoate', 'Thymidine', 'Reduced.riboflavin', 'Riboflavin', 'X2.Hydroxyphenylacetate', 'Sulfite', 'thiosulfate.2..', 'X.3.4.dihydroxyphenyl.acetate', 'Diphosphate', 'trimethylamine', 'glycine.betaine', 'Choline', 'Trimethylamine.N.oxide', 'Isochorismate', 'Inosine', 'xanthosine', 'Hypoxanthine', 'Xanthine', 'uracil', 'Uridine', 'Methanethiol', 'keto.phenylpyruvate', 'D.Arabinose', 'D.Galactose', 'Nicotinate', 'chorismate', 'tetradecanoate..n.C14.0.', 'oxalate.2..', 'X.R..3.hydroxybut

### Merge genetic data 
for training

In [15]:
# Create subject_id column by modifying sample_id
micom_func_tax['subject_id'] = micom_func_tax['sample_id'].str.split('.').str[0]  # Remove '.' and following characters
print("Subject_id column to match that of genetic data: ", micom_func_tax['subject_id'])

Subject_id column to match that of genetic data:  0      EBE-130
1      BSA-174
2      TDU-086
3      LVA-017
4      VCA-041
        ...   
294    QNG-166
295    NDI-067
296    TRO-113
297    SCA-043
298    NPO-149
Name: subject_id, Length: 299, dtype: object


In [16]:
print("gen_train columns: ", gen_train.columns.to_list())
gen_train = gen_train[['subject_id', 'bmi_prs']]
print("gen train subject_id: ", gen_train['subject_id'])

gen_train columns:  ['Unnamed: 0', 'subject_id', 'bmi_prs', 'old_or_new', 'predicted_BL_BMI', 'differences_BL_BMI', 'diff_BMI_quartile', 'diff_BMI_std', 'methyl_bmi_rs', 'methyl_bmi_rs_standardized']
gen train subject_id:  0      ABR-079
1      ACO-053
2      ADA-105
3      AGA-071
4      AHE-055
        ...   
115    TSH-146
116    TSL-056
117    VCA-041
118    YOR-103
119    YSU-097
Name: subject_id, Length: 120, dtype: object


In [17]:
merged_data = pd.merge(micom_func_tax, 
                       gen_train, 
                       on='subject_id', 
                       how='left')

print("Merged data shape: ", merged_data.shape)
print("Merged data columns: ", merged_data.columns.to_list())
# Count unique subject_id values
unique_subject_ids_gen = gen_train['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_gen)

unique_subject_ids_count = merged_data['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_count)

Merged data shape:  (308, 550)
Merged data columns:  ['Unnamed: 0_x', 'sample_id', 'proton', 'Diacetyl', 'Butyrate', 'Formate', 'acetone', 'X2.methylbutyrate', 'Valeric.Acid', 'Methane', 'X1.4.Dihydroxy.2.naphthoate', 'X2.Oxoglutarate', 'Pyruvate', 'D.lactate', 'L.lactate', 'laurate', 'Phenol', 'L.Carnitine', 'Isopropanol', 'p.Cresol', 'O2', 'X1.Propanol', 'butanol', 'Propanal', 'X4.hydroxyphenylacetate', 'X.S..propane.1.2.diol', 'Propane.1.3.diol', 'R_R__2_3_Butanediol', 'Oxaloacetate', 'sulfate', 'octadecenoate..n.C18.1.', 'o.Succinylbenzoate', 'Thymidine', 'Reduced.riboflavin', 'Riboflavin', 'X2.Hydroxyphenylacetate', 'Sulfite', 'thiosulfate.2..', 'X.3.4.dihydroxyphenyl.acetate', 'Diphosphate', 'trimethylamine', 'glycine.betaine', 'Choline', 'Trimethylamine.N.oxide', 'Isochorismate', 'Inosine', 'xanthosine', 'Hypoxanthine', 'Xanthine', 'uracil', 'Uridine', 'Methanethiol', 'keto.phenylpyruvate', 'D.Arabinose', 'D.Galactose', 'Nicotinate', 'chorismate', 'tetradecanoate..n.C14.0.', 'ox

In [18]:
# Get subject_ids from both DataFrames
gen_train_subject_ids = set(gen_train['subject_id'])
meta_train_subject_ids = set(train_long['subject_id'])
micom_func_tax_subject_ids = set(micom_func_tax['subject_id'])

# Find subject_ids in meta not in genetic data
subject_ids_in_meta_not_in_gen = meta_train_subject_ids - gen_train_subject_ids
print("Subject IDs in gen_train but not in micom_func_tax:", subject_ids_in_meta_not_in_gen)

subject_ids_in_meta_not_in_merged = meta_train_subject_ids - micom_func_tax_subject_ids
print("Subject IDs in meta but not in micom_func_tax:", subject_ids_in_meta_not_in_merged)

# Find subject_ids in gen_train but not in micom_func_tax
subject_ids_in_gen_not_in_micom = gen_train_subject_ids - micom_func_tax_subject_ids
print("Subject IDs in gen_train but not in micom_func_tax:", subject_ids_in_gen_not_in_micom)

# Find subject_ids in micom_func_tax but not in gen_train
subject_ids_in_micom_not_in_gen = micom_func_tax_subject_ids - gen_train_subject_ids
print("Subject IDs in micom_func_tax but not in gen_train:", subject_ids_in_micom_not_in_gen)

Subject IDs in gen_train but not in micom_func_tax: set()
Subject IDs in meta but not in micom_func_tax: {'MBA-176'}
Subject IDs in gen_train but not in micom_func_tax: {'MBA-176'}
Subject IDs in micom_func_tax but not in gen_train: {'NAR-099', 'NEL-094', 'ALI-121', 'CWA-183'}


#### For testing 

In [19]:
print("gen_test columns: ", gen_test.columns.to_list())
gen_test = gen_test[['subject_id', 'bmi_prs']]
print("gen train subject_id: ", gen_test['subject_id'])

gen_test columns:  ['Unnamed: 0', 'subject_id', 'bmi_prs', 'old_or_new', 'predicted_BL_BMI', 'differences_BL_BMI', 'diff_BMI_quartile', 'diff_BMI_std', 'methyl_bmi_rs', 'methyl_bmi_rs_standardized']
gen train subject_id:  0     AAL-144
1     AME-157
2     AWI-167
3     BHO-014
4     BHO-014
5     CAL-074
6     CBO-004
7     CEL-073
8     CIS-077
9     COW-066
10    CWA-161
11    DSC-024
12    FWI-098
13    HDE-154
14    JDI-140
15    JJO-093
16    JPO-022
17    JVE-126
18    KPA-042
19    LJA-101
20    LZD-142
21    MBA-187
22    MCW-065
23    MDI-107
24    MJA-153
25    MLU-106
26    MWE-143
27    NKA-090
28    NMO-151
29    SKA-087
30    WPE-005
31    ZVU-096
Name: subject_id, dtype: object


In [20]:
# Create subject_id column by modifying sample_id
test_micom_func_tax['subject_id'] = test_micom_func_tax['sample_id'].str.split('.').str[0]  # Remove '.' and following characters
print("Subject_id column to match that of genetic data: ", test_micom_func_tax['subject_id'])

Subject_id column to match that of genetic data:  0     JPO-022
1     BHO-014
2     JJO-093
3     COW-066
4     NMO-151
       ...   
65    WPE-005
66    CWA-161
67    BHO-014
68    FWI-098
69    AAL-144
Name: subject_id, Length: 70, dtype: object


In [21]:
test_merged_data = pd.merge(test_micom_func_tax, 
                       gen_test, 
                       on='subject_id', 
                       how='left')

print("Merged data shape: ", test_merged_data.shape)
print("Merged data columns: ", test_merged_data.columns.to_list())
# Count unique subject_id values
unique_subject_ids_gen = gen_test['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_gen)

unique_subject_ids_count = test_merged_data['subject_id'].nunique()
print("Number of unique subject_id values in merged: ", unique_subject_ids_count)

Merged data shape:  (73, 550)
Merged data columns:  ['Unnamed: 0_x', 'sample_id', 'proton', 'Diacetyl', 'Butyrate', 'Formate', 'acetone', 'X2.methylbutyrate', 'Valeric.Acid', 'Methane', 'X1.4.Dihydroxy.2.naphthoate', 'X2.Oxoglutarate', 'Pyruvate', 'D.lactate', 'L.lactate', 'laurate', 'Phenol', 'L.Carnitine', 'Isopropanol', 'p.Cresol', 'O2', 'X1.Propanol', 'butanol', 'Propanal', 'X4.hydroxyphenylacetate', 'X.S..propane.1.2.diol', 'Propane.1.3.diol', 'R_R__2_3_Butanediol', 'Oxaloacetate', 'sulfate', 'octadecenoate..n.C18.1.', 'o.Succinylbenzoate', 'Thymidine', 'Reduced.riboflavin', 'Riboflavin', 'X2.Hydroxyphenylacetate', 'Sulfite', 'thiosulfate.2..', 'X.3.4.dihydroxyphenyl.acetate', 'Diphosphate', 'trimethylamine', 'glycine.betaine', 'Choline', 'Trimethylamine.N.oxide', 'Isochorismate', 'Inosine', 'xanthosine', 'Hypoxanthine', 'Xanthine', 'uracil', 'Uridine', 'Methanethiol', 'keto.phenylpyruvate', 'D.Arabinose', 'D.Galactose', 'Nicotinate', 'chorismate', 'tetradecanoate..n.C14.0.', 'oxa

### merge meta data to other omics 

for training

In [22]:
print("Meta train columns: ", met_train.columns.to_list())
print("Meta train subject_id column: ", met_train['subject_id'])
print("test_micom_func_tax subject_id column: ", merged_data['subject_id'])

merged_data["time"] = merged_data["time"].astype(int)
met_train["time"] = met_train["time"].astype(int)
print("test_micom_func_tax time column: ", merged_data['time'])
print("met_train time column: ", met_train['time'])

# Count matching subject_id values
matching_subjects = merged_data['subject_id'].isin(met_train['subject_id']).sum()
print("Number of matching subject_id values:", matching_subjects)


Meta train columns:  ['Unnamed: 0', 'record_id', 'subject_id', 'randomized_group', 'cohort_number', 'sex', 'race', 'age', 'outcome_BMI_fnl', 'Glucose', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid', 'Peptide_YY', 'Ghrelin', 'Leptin', 'Hemoglobin_A1C', 'time']
Meta train subject_id column:  0      LFI-003
1      ROL-006
2      AKE-009
3      HGI-010
4      AKI-011
        ...   
286    SKA-195
287    KHU-196
288    LPF-198
289    KEL-199
290    BIN-201
Name: subject_id, Length: 291, dtype: object
test_micom_func_tax subject_id column:  0      EBE-130
1      BSA-174
2      TDU-086
3      LVA-017
4      VCA-041
        ...   
303    NDI-067
304    TRO-113
305    SCA-043
306    SCA-043
307    NPO-149
Name: subject_id, Length: 308, dtype: object
test_micom_func_tax time column:  0       0
1       0
2       0
3       0
4       0
       ..
303    12
304    12
305    12
306    12
307    12
Name: time, Length: 308, dtype: int64
met_train time column

In [23]:
merged_all = pd.merge(met_train, merged_data, 
                         left_on=['subject_id', 'time'], 
                         right_on=['subject_id', 'time'], 
                         how='inner',
                         suffixes=('_train_long', '_merged_data'))

print("Merged all shape: ", merged_all.shape)
print("Merged all columns: ", merged_all.columns.to_list())
# Count unique subject_id values
unique_subject_ids_all = merged_all['subject_id'].nunique()
print("Number of unique subject_id values in merged_all: ", unique_subject_ids_all)

Merged all shape:  (280, 568)
Merged all columns:  ['Unnamed: 0_train_long', 'record_id', 'subject_id', 'randomized_group', 'cohort_number', 'sex', 'race', 'age', 'outcome_BMI_fnl', 'Glucose', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid', 'Peptide_YY', 'Ghrelin', 'Leptin', 'Hemoglobin_A1C', 'time', 'Unnamed: 0_x', 'sample_id', 'proton', 'Diacetyl', 'Butyrate', 'Formate', 'acetone', 'X2.methylbutyrate', 'Valeric.Acid', 'Methane', 'X1.4.Dihydroxy.2.naphthoate', 'X2.Oxoglutarate', 'Pyruvate', 'D.lactate', 'L.lactate', 'laurate', 'Phenol', 'L.Carnitine', 'Isopropanol', 'p.Cresol', 'O2', 'X1.Propanol', 'butanol', 'Propanal', 'X4.hydroxyphenylacetate', 'X.S..propane.1.2.diol', 'Propane.1.3.diol', 'R_R__2_3_Butanediol', 'Oxaloacetate', 'sulfate', 'octadecenoate..n.C18.1.', 'o.Succinylbenzoate', 'Thymidine', 'Reduced.riboflavin', 'Riboflavin', 'X2.Hydroxyphenylacetate', 'Sulfite', 'thiosulfate.2..', 'X.3.4.dihydroxyphenyl.acetate', 'Diphosphate', 

#### For testing 

In [24]:
test_merged_all = pd.merge(met_test, 
                       test_merged_data, 
                        left_on=['subject_id', 'time'], 
                        right_on=['subject_id', 'time'],
                        how='inner',
                        suffixes=('_test_long', '_merged_data'))  # Specify suffixes for duplicate columns

print("Merged all shape: ", test_merged_all.shape)
print("Merged all columns: ", test_merged_all.columns.to_list())
# Count unique subject_id values
unique_subject_ids_all = test_merged_all['subject_id'].nunique()
print("Number of unique subject_id values in merged_all: ", unique_subject_ids_all)

Merged all shape:  (66, 568)
Merged all columns:  ['Unnamed: 0_test_long', 'record_id', 'subject_id', 'randomized_group', 'cohort_number', 'sex', 'race', 'age', 'outcome_BMI_fnl', 'Glucose', 'HOMA_IR', 'Insulin_endo', 'HDL_Total_Direct_lipid', 'LDL_Calculated', 'Triglyceride_lipid', 'Peptide_YY', 'Ghrelin', 'Leptin', 'Hemoglobin_A1C', 'time', 'Unnamed: 0_x', 'sample_id', 'proton', 'Diacetyl', 'Butyrate', 'Formate', 'acetone', 'X2.methylbutyrate', 'Valeric.Acid', 'Methane', 'X1.4.Dihydroxy.2.naphthoate', 'X2.Oxoglutarate', 'Pyruvate', 'D.lactate', 'L.lactate', 'laurate', 'Phenol', 'L.Carnitine', 'Isopropanol', 'p.Cresol', 'O2', 'X1.Propanol', 'butanol', 'Propanal', 'X4.hydroxyphenylacetate', 'X.S..propane.1.2.diol', 'Propane.1.3.diol', 'R_R__2_3_Butanediol', 'Oxaloacetate', 'sulfate', 'octadecenoate..n.C18.1.', 'o.Succinylbenzoate', 'Thymidine', 'Reduced.riboflavin', 'Riboflavin', 'X2.Hydroxyphenylacetate', 'Sulfite', 'thiosulfate.2..', 'X.3.4.dihydroxyphenyl.acetate', 'Diphosphate', 't

In [25]:
# Remove specified columns from merged_all
merged_all = merged_all.drop(columns=['Unnamed: 0_train_long', 'Unnamed: 0_x', 'all_samples_x', 'time_x', 
                                       'Unnamed: 0_y', 'SampleID_x', 'SampleID_y'])

# Remove specified columns from test_merged_all
test_merged_all = test_merged_all.drop(columns=['Unnamed: 0_test_long', 'Unnamed: 0_x', 'all_samples_x', 'time_x', 
                                               'Unnamed: 0_y', 'SampleID_x', 'SampleID_y'])

# Save combined training and testing files 

In [26]:
#merged_all.to_csv(os.path.join(df_dir, 'feb20_training_merged_all_omics_raw_meta.csv'), index=False)  # Save merged_all to CSV
#test_merged_all.to_csv(os.path.join(df_dir, 'feb20_test_merged_all_omics_raw_meta.csv'), index=False)  # Save test_merged_all to CSV

merged_all.to_csv(os.path.join(df_dir, 'feb20_training_merged_all_omics_extra_meta.csv'), index=False)  # Save merged_all to CSV
test_merged_all.to_csv(os.path.join(df_dir, 'feb20_test_merged_all_omics_extra_meta.csv'), index=False)